In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
from coralme.builder.main import MEBuilder

builder = MEBuilder(*['iJL1678b-ME/coralme-config-tRNAs-TUs.yaml'])
builder.configuration['run_bbh_blast'] = False
builder.configuration['feature_types'] = ['CDS', 'rRNA', 'tRNA', 'ncRNA']
# builder.configuration['gam'] = 0.
# builder.configuration['ngam'] = 0.
builder.build_me_model(overwrite = False)
# builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

/home/jt/UCSD/coralme/coralme/builder/main.py:30: UserWarning: This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.
  warnings.warn("This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.")
/home/jt/UCSD/coralme/coralme/builder/preprocess_inputs.py:15: UserWarning: This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.
  warnings.warn("This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.")


Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-12
Initiating ME-model reconstruction...


Adding biomass constraint(s) into the ME-model...                          : 100.0%|████████████████████████████████████████|    11/   11 [00:00<00:00]


Read LP format model from file /tmp/tmptirj5bq1.lp
Reading time = 0.02 seconds
: 1660 rows, 4754 columns, 17902 nonzeros


Adding Metabolites from M-model into the ME-model...                       : 100.0%|████████████████████████████████████████|  1810/ 1810 [00:00<00:00]
Adding Reactions from M-model into the ME-model...                         : 100.0%|████████████████████████████████████████|  2610/ 2610 [00:00<00:00]
Adding Transcriptional Units into the ME-model...                          : 100.0%|████████████████████████████████████████|  3415/ 3415 [01:49<00:00]
Adding features from contig NC_000913.2 into the ME-model...               : 100.0%|████████████████████████████████████████|  4317/ 4317 [00:53<00:00]
Adding features from contig b0235 into the ME-model...                     : 100.0%|████████████████████████████████████████|     2/    2 [00:00<00:00]
Adding features from contig b4644 into the ME-model...                     : 100.0%|████████████████████████████████████████|     2/    2 [00:00<00:00]
Adding features from contig b0691 into the ME-model...                     : 100.0%|████

ME-model was saved in the iJL1678b-ME directory as MEModel-step1-iJL1678b-ME.pkl


Adding tRNA synthetase(s) information into the ME-model...                 : 100.0%|████████████████████████████████████████|   158/  158 [00:00<00:00]
Adding tRNA modification SubReactions...                                   : 100.0%|████████████████████████████████████████|    34/   34 [00:00<00:00]
Associating tRNA modification enzyme(s) to tRNA(s)...                      : 100.0%|████████████████████████████████████████|   650/  650 [00:03<00:00]
Adding SubReactions into TranslationReactions...                           : 100.0%|████████████████████████████████████████|  4279/ 4279 [00:09<00:00]
Adding RNA Polymerase(s) into the ME-model...                              : 100.0%|████████████████████████████████████████|     7/    7 [00:00<00:00]
Associating a RNA Polymerase to each Transcriptional Unit...               : 100.0%|████████████████████████████████████████|  3415/ 3415 [00:00<00:00]
Processing ComplexData in ME-model...                                      : 100.0%|████

ME-model was saved in the iJL1678b-ME directory as MEModel-step2-iJL1678b-ME.pkl
ME-model reconstruction is done.
Number of metabolites in the ME-model is 7197 (+333.55%, from 1660)
Number of reactions in the ME-model is 14959 (+529.32%, from 2377)
Number of genes in the ME-model is 1689 (+32.99%, from 1270)


In [4]:
with open('genes.txt', 'r') as infile:
    data = pandas.read_fwf(infile, header = None)
iJLgenes = data.iloc[:, 0].to_list()

import re
me_genes = [ x.id.split('RNA_')[1] for x in builder.me_model.metabolites.query(re.compile('^RNA_(?!biomass|dummy|degradosome)')) ]

genes = sorted(set(me_genes).difference(iJLgenes))
print('Genes in CORAL-ME, not in ECOLI-ME: {:d} ({:s})'.format(len(genes), ','.join(genes)))
print()

genes = sorted(set(iJLgenes).difference(me_genes))
print('Genes in ECOLI-ME, not in CORAL-ME: {:d} ({:s})'.format(len(genes), ','.join(genes)))

Genes in CORAL-ME, not in ECOLI-ME: 13 (b0195,b1807,b2583,b2726,b2727,b2728,b2729,b2730,b3064,b3414,b3497,b4168,b4373)

Genes in ECOLI-ME, not in CORAL-ME: 2 (b2710,b2711)


In [ ]:
import pickle, tqdm
with open('./iJL1678b-ME/MEModel-step2-iJL1678b-ME.pkl', 'rb') as infile:
    me = pickle.load(infile)
    
with open('./MEModel-step2-ecoli.pkl','wb') as f:
    pickle.dump(me, f)

me.add_boundary(me.metabolites.li_c, type = 'sink')

import coralme
coralme.io.json.save_json_me_model(me, 'MEModel-step2-ecoli.json')

# me.optimize()

In [ ]:
with open('ecolime-fluxes-from-model.txt', 'r') as infile:
    data = pandas.read_csv(infile, sep = '\t', header = None)
iJLreactions = data.iloc[:, 0].to_list()

import re
me_reactions = [ x.id for x in me.reactions if x.bounds != (0, 0) ]
me_reactions = [ x.replace('_Outer_Membrane', '') if x.startswith('translocation') else x for x in me_reactions ]
me_reactions = [ x.replace('_Inner_Membrane', '') if x.startswith('translocation') else x for x in me_reactions ]
me_reactions = [ x.replace('_Periplasm', '') if x.startswith('translocation') else x for x in me_reactions ]

reactions = sorted(set(me_reactions).difference(iJLreactions))
print('Reactions in CORAL-ME, not in ECOLI-ME: {:d} ({:s})'.format(len(reactions), ''))#, ','.join(genes)))
print()

reactions = sorted(set(iJLreactions).difference(me_reactions))
print('Reactions in ECOLI-ME, not in CORAL-ME: {:d} ({:s})'.format(len(reactions), ''))#, ','.join(genes)))

with open('mismatching_reactions.txt', 'w') as outfile:
    for data in sorted([ x for x in set(iJLreactions).symmetric_difference(me_reactions) 
                        if not x.startswith('transcription_') if not x.startswith('translocation_') ]):
        outfile.write(data)
        outfile.write('\n')
        
with open('mismatching_reactions_transcription_only.txt', 'w') as outfile:
    for data in sorted([ x for x in set(iJLreactions).symmetric_difference(me_reactions) 
                        if x.startswith('transcription_') ]):
        outfile.write(data)
        outfile.write('\n')
        
with open('mismatching_reactions_translocation_only.txt', 'w') as outfile:
    for data in sorted([ x for x in set(iJLreactions).symmetric_difference(me_reactions) 
                        if x.startswith('translocation_') ]):
        outfile.write(data)
        outfile.write('\n')

In [ ]:
me.optimize()

In [ ]:
2

In [ ]:
2